In [1]:
#%source bin/activate
import tensorflow as tf
import os
import skimage.io as io
import time

In [2]:
IMG_HEIGHT, IMG_WIDTH = 128, 128

# labels -- [iso, D, T, X, U, L]
NLBL = 2

rundir = "/home/walterms/project/walterms/mcmd/nn/cnn/data/edgevar/"
testdir = rundir+"test/"
unlbldir = rundir+"unlbl/"
trndir = rundir+"train/"

testsubset = ["edge15.tfrecords", "edge40.tfrecords"]
unlblsubset = [""]
trnsubset = ["edge15.tfrecords", "edge40.tfrecords"]

testrecords, unlblrecords, trnrecords = [],[],[]
if testsubset == [""]:
    testrecords = [testdir+x for x in os.listdir(testdir)]
else:
    testrecords = [testdir+x for x in testsubset]
if trnsubset == [""]:
    trnrecords = [trndir+x for x in os.listdir(trndir)]
else:
    trnrecords = [trndir+x for x in trnsubset]
if unlblsubset == [""]:
    unlblrecords = [unlbldir+x for x in os.listdir(unlbldir)]
else:
    unlblrecords = [unlbldir+x for x in unlblsubset]

In [6]:
def read_records(filename_queue, batchSize, labeled):
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    
    features = None
    if labeled == True:
        features = tf.parse_single_example(
          serialized_example,
          # Defaults are not specified since both keys are required.
          features={
            'height': tf.FixedLenFeature([], tf.int64),
            'width': tf.FixedLenFeature([], tf.int64),
            'image_raw': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64)
            })
    elif labeled == False:
        features = tf.parse_single_example(
          serialized_example,
          # Defaults are not specified since both keys are required.
          features={
            'height': tf.FixedLenFeature([], tf.int64),
            'width': tf.FixedLenFeature([], tf.int64),
            'image_raw': tf.FixedLenFeature([], tf.string)
            })
    else:
        print "Need to specify record type (labeled or not) for read_records"
        return

    # Convert from a scalar string tensor (whose single string has
    # length mnist.IMAGE_PIXELS) to a uint8 tensor with shape
    # [mnist.IMAGE_PIXELS].
    image = tf.decode_raw(features['image_raw'], tf.uint8)
#     image = tf.decode_raw(features['image_raw'], tf.uint8)
#     height = tf.cast(features['height'], tf.int32)
#     width = tf.cast(features['width'], tf.int32)
    
    label = None
    blankvec = None
    if labeled == True:
        label = tf.cast(features['label'], tf.int32)
        v = [0 for i in range(NLBL)]
        blankvec = tf.Variable(v)    
    
#     image_shape = tf.stack([128,128])
    image_shape = tf.stack([IMG_HEIGHT,IMG_WIDTH])
    image_reshaped = tf.reshape(image, image_shape)
#     image_size_const = tf.constant((IMG_HEIGHT, IMG_WIDTH,1), dtype=tf.int32)
    
#     # Random transformations can be put here: right before you crop images
#     # to predefined size. To get more information look at the stackoverflow
#     # question linked above.
    
#     resized_image = tf.image.resize_image_with_crop_or_pad(image=image,
#                                            target_height=IMG_HEIGHT,
#                                            target_width=IMG_WIDTH)
    
    if labeled == True:
        return resized_image, label, blankvec
    else:
        return resized_image

def shuffled_queue_join(Qlist, batchSize):
    return tf.train.shuffle_batch_join(Qlist,
                                        batch_size=batchSize,
                                        capacity=1000+3*batchSize,
                                        min_after_dequeue=1000)

In [7]:
filterWidth = 15
stride = 8
conv1_nOut = 32
conv2_nOut = 64
fc1_nIn = IMG_HEIGHT*IMG_WIDTH*conv2_nOut/(stride*stride*stride*stride)
fc1_nOut = 512

trnBatchSize = 100
testSetSize = 200
eta = 1e-4
nTrnIter = 2000
testStep = nTrnIter/20

beta = 1e-3

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    s = stride
    return tf.nn.conv2d(x, W, strides=[1,s,s,1], padding='SAME')

def assign_labels():
    return

x = tf.placeholder(tf.float32, shape=[None, IMG_HEIGHT, IMG_WIDTH])
# x = tf.placeholder(tf.float32, shape=[None, IMG_HEIGHT, IMG_WIDTH])
x_norm = tf.scalar_mul(1./255.,x)
y_ = tf.placeholder(tf.float32, shape=[None, NLBL])

x_image = tf.reshape(x_norm, [-1, IMG_HEIGHT, IMG_WIDTH, 1])

W_conv1 = weight_variable([filterWidth, filterWidth, 1, conv1_nOut]) # [x,y,nInputChannel,nOutChannel]
b_conv1 = bias_variable([conv1_nOut])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

W_conv2 = weight_variable([filterWidth, filterWidth, conv1_nOut, conv2_nOut])
b_conv2 = bias_variable([conv2_nOut])
h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2) + b_conv2)

# Dividing by stride*stride for each conv layer
W_fc1 = weight_variable([fc1_nIn, fc1_nOut])
b_fc1 = bias_variable([fc1_nOut])
h_conv2_flat = tf.reshape(h_conv2, [-1, fc1_nIn])
h_fc1 = tf.nn.relu(tf.matmul(h_conv2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([fc1_nOut, NLBL])
b_fc2 = bias_variable([NLBL])

y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

regularizers = tf.nn.l2_loss(W_fc2) + tf.nn.l2_loss(W_fc1)
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
loss = tf.reduce_mean(cross_entropy + beta * regularizers)
train_step = tf.train.AdamOptimizer(eta).minimize(loss)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# the Qlist is needed for shuffling
trn_queue = tf.train.string_input_producer(trnrecords)
trnQlist = [read_records(trn_queue, trnBatchSize, labeled=True) 
            for _ in range(len(trnrecords))]
# Even when reading in multiple threads, share the filename queue
trnbatch = shuffled_queue_join(trnQlist, trnBatchSize)

# make a Qlist for test in case we only want
# to test a subset
test_queue = tf.train.string_input_producer(testrecords)
testQlist = [read_records(test_queue, testSetSize, labeled=True) 
             for _ in range(len(trnrecords))]
testbatch = shuffled_queue_join(testQlist, testSetSize)

unlbl_queue = tf.train.string_input_producer([unlblrecords[0]], num_epochs=1)

# The op for initializing the variables.
init_op = tf.group(tf.global_variables_initializer(),
                   tf.local_variables_initializer())

print "About to start sess"
beg_ts = time.time()
chkpt_ts = time.time()

with tf.Session() as sess:
    
    sess.run(init_op)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    print "Initialized sess"
    
    for i in xrange(0,nTrnIter):
    
        trnbatch_py = sess.run([trnbatch[0], trnbatch[1], trnbatch[2]])
        
        # Assign proper labels to the lblvecs
        # Currently lblvecs is a batch of blank (zeros) rows
        # labels -- [iso, D, T, X, U, L]
        for l in range(trnBatchSize):
            trnbatch_py[2][l][trnbatch_py[1][l]] = 1

        train_step.run(feed_dict={x: trnbatch_py[0], y_: trnbatch_py[2], keep_prob: 0.5})
        
#         print trnbatch_py[2][0:4]
#         for p in trnbatch_py[0][0:4]:
#             io.imshow(p)
#             io.show()
        now = time.time()
        if (now-chkpt_ts) > 60:
            print(">> elapsed time: %f" % (now - beg_ts))
            chkpt_ts = now
        if i % testStep == 0:
            trn_accuracy = accuracy.eval(feed_dict={
                x: trnbatch_py[0], y_: trnbatch_py[2], keep_prob: 1.0})

            testbatch_py = sess.run([testbatch[0], testbatch[1], testbatch[2]])
            testLblSums = [0 for b in range(NLBL)]
            for l in range(testSetSize):
                testbatch_py[2][l][testbatch_py[1][l]] = 1
                testLblSums[testbatch_py[1][l]] += 1
                
            test_accuracy = accuracy.eval(feed_dict={
                x: testbatch_py[0], y_: testbatch_py[2], keep_prob: 1.0})
            print('step %d, training accuracy (sample) %g, test accuracy %g' 
                  % (i, trn_accuracy, test_accuracy))
            print "Test Label Sums:", testLblSums
        
    print "Closing sess"
    coord.request_stop()
    coord.join(threads)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

ValueError: 'image' must have either 3 or 4 dimensions.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
trnbatch_py[0][0].shape

In [ ]:
len(trnbatch_py[0])

In [ ]:
for i in range(200):
    print np.min(trnbatch_py[0][i])

In [ ]:
im = 1./255*trnbatch_py[0][0]

In [ ]:
minn = np.min(im)
maxx = np.max(im)
f = 1/(maxx-minn)
im = f*(im-minn)
print np.min(im), np.max(im)

In [ ]:
im.shape

In [ ]:
img = plt.imshow(im)